In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import re
from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('retail_train.csv')
result = pd.read_csv('predictions_basic.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### **Задание 1. Weighted Random Recommendation**

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity['weight'] = popularity['sales_value'] / popularity['sales_value'].sum()
popularity.head(2)

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False)
    
    return recs.tolist()

In [6]:
%%time

# your_code

items_weights = popularity[['item_id', 'weight']]

result_weighted = pd.DataFrame()
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 2.89 s, sys: 625 µs, total: 2.89 s
Wall time: 2.9 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1075368, 909972, 1054262, 916381, 1124081]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[872137, 1045288, 1040807, 5584007, 1030819]"


### **Задание 2. Расчет метрик**
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [8]:
# функция для преобразования строк в список
def str_to_array(result):
    shape_ = result.shape[0]
    for col in result.columns[1:]:        
        for string in range(shape_):
            if isinstance(result[col].iloc[string], str):
                result[col].iloc[string] = [int(_) for _ in re.findall(r'-?\d+\.?\d*', result[col].iloc[string])]
            else:
                result[col].iloc[string] = result[col].iloc[string]

    return result

In [9]:
str_to_array(result).head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1075368, 909972, 1054262, 916381, 1124081]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[872137, 1045288, 1040807, 5584007, 1030819]"


In [10]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0214:  weighted_random_recommendation


### **Задание 3*. Улучшение бейзлайнов и ItemItem**

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [12]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [13]:
# отберем топ-5000
quant_sum = data_train.groupby('item_id')['quantity'].sum().reset_index()
quant_sum.sort_values(by='quantity', ascending=False, inplace=True)
top_5000 = quant_sum.head(5000)

In [14]:
%%time

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000['item_id'], n=5))
result.head(2)

CPU times: user 266 ms, sys: 851 µs, total: 267 ms
Wall time: 266 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1075368, 909972, 1054262, 916381, 1124081]","[1138443, 1124077, 1096643, 9368132, 5565352]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[872137, 1045288, 1040807, 5584007, 1030819]","[1033142, 9526666, 996444, 12731641, 912681]"


In [15]:
%%time

data_train_top5000 = data_train.loc[data_train['item_id'].isin(top_5000['item_id']), :]
result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train_top5000, n=5))
result.head(2)

CPU times: user 1min 10s, sys: 95.9 ms, total: 1min 10s
Wall time: 1min 10s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1075368, 909972, 1054262, 916381, 1124081]","[1138443, 1124077, 1096643, 9368132, 5565352]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[872137, 1045288, 1040807, 5584007, 1030819]","[1033142, 9526666, 996444, 12731641, 912681]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [16]:
data_train_top5000.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [17]:
popularity_5000 = data_train_top5000.groupby('item_id')['sales_value'].sum().reset_index()
popularity_5000['weight'] = popularity_5000['sales_value'] / popularity_5000['sales_value'].sum()
popularity_5000.head(2)

,item_id,sales_value,weight
0,202291,81.59,0.000020
1,397896,2862.41,0.000706


In [18]:
%%time

items_weights = popularity_5000[['item_id', 'weight']]
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 364 ms, sys: 999 µs, total: 365 ms
Wall time: 365 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5000,popular_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1075368, 909972, 1054262, 916381, 1124081]","[1138443, 1124077, 1096643, 9368132, 5565352]","[6534178, 6533889, 1029743, 6534166, 1082185]","[991592, 6534178, 889509, 995242, 1008625]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[872137, 1045288, 1040807, 5584007, 1030819]","[1033142, 9526666, 996444, 12731641, 912681]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1129610, 8090537, 879528, 885863, 825994]"


In [19]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0214:  weighted_random_recommendation
0.0058:  random_recommendation_5000
0.1552:  popular_recommendation_5000
0.0369:  weighted_random_recommendation_5000


In [20]:
compare_table_5000 = pd.DataFrame({
    'precision_at_k': [0.0006, 0.006, 0.1552, 0.1552, 0.0212, 0.0374],
    'name_col': ['random_recommendation', 'random_recommendation_5000', 'popular_recommendation', 'popular_recommendation_5000', \
                 'weighted_random_recommendation', 'weighted_random_recommendation_5000']   
})

compare_table_5000

,precision_at_k,name_col
0,0.0006,random_recommendation
1,0.0060,random_recommendation_5000
2,0.1552,popular_recommendation
3,0.1552,popular_recommendation_5000
4,0.0212,weighted_random_recommendation
5,0.0374,weighted_random_recommendation_5000


In [21]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000['item_id']), 'item_id'] = 999999
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,999999,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [22]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [23]:
%%time

k_num = [3, 5, 7]
result_recs = []

for i in k_num:
    
    model = ItemItemRecommender(K=i, num_threads=2) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)
    recs = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    
    result_recs.append(recs)
    
    
result['itemitem_3'] = result_recs[0]
result['itemitem_5'] = result_recs[1]
result['itemitem_7'] = result_recs[2]

CPU times: user 4.59 s, sys: 8.79 ms, total: 4.6 s
Wall time: 2.93 s


In [24]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0214:  weighted_random_recommendation
0.0058:  random_recommendation_5000
0.1552:  popular_recommendation_5000
0.0369:  weighted_random_recommendation_5000
0.1861:  itemitem_3
0.1369:  itemitem_5
0.1449:  itemitem_7


In [25]:
compare_table_item_k = pd.DataFrame({
    'precision_at_k': [0.1369, 0.1861, 0.1369, 0.1449],
    'name_col': ['itemitem', 'itemitem_3', 'itemitem_5', 'itemitem_7']   
})

compare_table_item_k

,precision_at_k,name_col
0,0.1369,itemitem
1,0.1861,itemitem_3
2,0.1369,itemitem_5
3,0.1449,itemitem_7
